In [1]:
import cv2 as cv
import numpy as np
import os
import sklearn
from sklearn.svm import SVC
from skimage.feature import hog
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV ,StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

import pickle

In [2]:
MASK_PATH = './Dataset/mask/'
NO_MASK_PATH = './Dataset/no_mask/'

MASK_PATH_PREPROCESSED = './PreProcessed/mask/'
NO_MASK_PATH_PREPROCESSED = './PreProcessed/no_mask/'


In [3]:
mask_data = []
no_mask_data = []

for i in os.listdir(MASK_PATH_PREPROCESSED):
    path = os.path.join(MASK_PATH_PREPROCESSED,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)
    mask_data.append(img)
   
for i in os.listdir(NO_MASK_PATH_PREPROCESSED):
    path = os.path.join(NO_MASK_PATH_PREPROCESSED,i)
    img = cv.imread(path,cv.IMREAD_GRAYSCALE)
    no_mask_data.append(img)

In [4]:
# cv.imshow('',no_mask_data[0])
# cv.waitKey(0)
# cv.destroyAllWindows()

In [5]:
masked_features = []
no_masked_features = []

for i in mask_data:
    hog_feature = hog(i, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    masked_features.append(hog_feature)
    
for i in no_mask_data:
    hog_feature = hog(i, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    no_masked_features.append(hog_feature)

In [6]:
df = pd.DataFrame(masked_features)
df['label'] = 1

df1 = pd.DataFrame(no_masked_features)
df1['label'] = 0

merged = pd.concat([df,df1])
merged

,0,1,2,3,4,5,6,7,8,9,...,8091,8092,8093,8094,8095,8096,8097,8098,8099,label
0,0.256592,0.150881,0.063293,0.110772,0.192694,0.035487,0.088102,0.172904,0.256592,0.256592,...,0.241442,0.088172,0.019063,0.020524,0.113308,0.013188,0.044941,0.087931,0.057706,1
1,0.201315,0.048795,0.076720,0.173655,0.253165,0.212584,0.253165,0.253165,0.166358,0.253165,...,0.343353,0.329742,0.157182,0.145159,0.102832,0.028763,0.045567,0.024396,0.171204,1
2,0.302393,0.302393,0.151530,0.077068,0.024507,0.012864,0.000000,0.055860,0.052579,0.302393,...,0.273294,0.009949,0.007654,0.028541,0.015255,0.040541,0.021648,0.293596,0.293596,1
3,0.144452,0.121186,0.224592,0.153852,0.203977,0.115354,0.262002,0.112103,0.238174,0.262459,...,0.235790,0.035626,0.047785,0.044443,0.235790,0.235790,0.235790,0.228595,0.171928,1
4,0.103517,0.029844,0.031505,0.034045,0.047750,0.000000,0.007235,0.021402,0.045056,0.343491,...,0.317624,0.111844,0.167507,0.136179,0.240884,0.006157,0.317624,0.092421,0.307481,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,0.126381,0.064085,0.061053,0.181789,0.274194,0.156359,0.126816,0.044444,0.022013,0.202070,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
638,0.476735,0.020055,0.000000,0.000000,0.007139,0.000000,0.000000,0.000000,0.051792,0.146354,...,0.260156,0.023596,0.000000,0.000000,0.066348,0.040199,0.061911,0.260156,0.260156,0
639,0.287913,0.287913,0.287913,0.036980,0.101450,0.093302,0.043555,0.027799,0.060805,0.217517,...,0.048719,0.004790,0.035910,0.188794,0.280518,0.280518,0.233272,0.050732,0.053052,0
640,0.257836,0.093657,0.051381,0.041461,0.257836,0.220667,0.257836,0.257836,0.257836,0.257836,...,0.233575,0.040427,0.077076,0.104611,0.150520,0.088142,0.087759,0.187778,0.159242,0


In [7]:
y = merged['label']
x = merged.drop(columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [8]:
svm = SVC(kernel='rbf',gamma=0.01, C=0.1)
svm.fit(X_train,y_train)


cross_validation = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, cv=cross_validation,refit = True, verbose = 3) 
  
grid.fit(X_train, y_train) 


Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.506, total=   3.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.506, total=   3.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.506, total=   3.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.506, total=   3.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.512, total=   3.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.512, total=   3.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .........

[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:  1.9min


[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.621, total=   3.9s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.667, total=   3.9s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.586, total=   3.8s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.605, total=   3.8s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.593, total=   3.7s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.640, total=   3.7s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] ...... C=0.1, gamma=0.001, kernel=rbf, score=0.628, total=   3.8s
[CV] C=0.1, gamma=0.001, kernel=rbf ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 127 tasks      | elapsed:  7.2min


[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.965, total=   3.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.977, total=   2.1s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ........ C=10, gamma=0.01, kernel=rbf, score=0.965, total=   2.9s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.989, total=   1.4s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.989, total=   1.4s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=1.000, total=   1.4s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ....... C=10, gamma=0.001, kernel=rbf, score=0.977, total=   1.4s
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] .

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [9]:
print(grid.best_params_)

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}


In [10]:
# classifier = SVC(C= 10, gamma= 0.01, kernel= 'rbf')
# classifier.fit(X_train, y_train)

In [15]:
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

ytrain_pred = best_model.predict(X_train)
ytest_pred = best_model.predict(X_test)


print(f"Best parameters found: {grid.best_params_}")
print(f"Best cross-validation score: {grid.best_score_}")

print(classification_report(y_train, ytrain_pred))
print(classification_report(y_test, ytest_pred))

Best parameters found: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation score: 0.9814621758887998
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       437
           1       1.00      1.00      1.00       427

    accuracy                           1.00       864
   macro avg       1.00      1.00      1.00       864
weighted avg       1.00      1.00      1.00       864

              precision    recall  f1-score   support

           0       0.94      0.99      0.96       205
           1       0.99      0.94      0.97       222

    accuracy                           0.96       427
   macro avg       0.97      0.97      0.96       427
weighted avg       0.97      0.96      0.96       427



In [12]:

matrix_kebingungan = confusion_matrix(ytest_pred, y_test)
print(matrix_kebingungan)

matrix_kebingungan = confusion_matrix(ytrain_pred, y_train)
print(matrix_kebingungan)

[[203  13]
 [  2 209]]
[[437   0]
 [  0 427]]


In [13]:




# img = cv.imread('./b.jpg', cv.IMREAD_GRAYSCALE)
# # img = no_mask_data[-1:]
# # print(img)
# faces = haarcasade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 5)
# if not len(faces) < 1:
#     for face_rect in faces:
#         x, y, w, h = face_rect
#         face_image = img[y:y+w, x:x+h]
#         resized = cv.resize(face_image,(128,128))

#         hog_feature = hog(resized, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
#         test_data = pd.DataFrame(hog_feature).T

#         result=best_model.predict(test_data)[0]
#         print(result)
#         if result == 0:
#             result = 'No Mask'
#         else:
#             result = 'Mask'


#         cv.putText(img, result, (100,100), cv.FONT_HERSHEY_PLAIN, 1.5, (255, 0, 0), 1)
#         cv.imshow('',img)
#         cv.waitKey(0)
#         cv.destroyAllWindows()

#         cv.imshow('',resized)
#         cv.waitKey(0)
#         cv.destroyAllWindows()

In [14]:
import pickle
model_filename = './model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(best_model, file)

print(f"Model saved to {model_filename}")

Model saved to ./model.pkl
